In [2]:
print("Hello")

Hello


# install libraries

In [11]:
# pip install apify-client
# !pip install textblob
# pip install Praw
# pip install TikTokApi 

# Extract data from reddddit

In [6]:
import praw

reddit = praw.Reddit(
    client_id="h7SCVUWMMkrKxa6dW7K6aA",
    client_secret="ndIj2lKITQBJ6NzHAD10FkyuPZer2A",
    user_agent="my_app"
)

# Example: fetch top posts from r/electronic_cigarette
subreddit = reddit.subreddit("maker")

for post in subreddit.top(limit=5):
    print({
        "datetime": post.created_utc,
        "niche": "vape",
        "product": None,
        "hashtags": None,
        "description": post.title,
        "country": None,
        "area": None,
        "store/channel_name": post.author.name if post.author else None,
        "social_platform": "reddit"
    })


{'datetime': 1754839769.0, 'niche': 'vape', 'product': None, 'hashtags': None, 'description': 'I built a machine that turns thread into portraits', 'country': None, 'area': None, 'store/channel_name': 'Quiet_Compote_6803', 'social_platform': 'reddit'}
{'datetime': 1462377072.0, 'niche': 'vape', 'product': None, 'hashtags': None, 'description': 'Maker AMA Series: I am Adam Savage, maker, editor-in-chief of Tested.com and former host of MythBusters. AMAAM! (Ask Me Anything About Making!)', 'country': None, 'area': None, 'store/channel_name': 'mistersavage', 'social_platform': 'reddit'}
{'datetime': 1742483895.0, 'niche': 'vape', 'product': None, 'hashtags': None, 'description': 'I made a really cool lamp and I’m really proud of it!', 'country': None, 'area': None, 'store/channel_name': 'careyi4', 'social_platform': 'reddit'}
{'datetime': 1740090113.0, 'niche': 'vape', 'product': None, 'hashtags': None, 'description': 'I made an eternal flame in a tiny dumpster that burns until the end of

# Extract relevant information only 

In [8]:
# ----- PRAW: targeted vape data fetch (last 7 days) -----
import re
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo

import praw

# --- 1) Configure your Reddit credentials ---
# reddit = praw.Reddit(
#     client_id="YOUR_CLIENT_ID",
#     client_secret="YOUR_CLIENT_SECRET",
#     user_agent="my_vape_app/1.0"
# )
reddit = praw.Reddit(
    client_id="h7SCVUWMMkrKxa6dW7K6aA",
    client_secret="ndIj2lKITQBJ6NzHAD10FkyuPZer2A",
    user_agent="my_app"
)

# --- 2) Target subreddits that are actually about vaping ---
VAPE_SUBS = [
         "VapeWild","Juul" # remove subs you don't want
]

# --- 3) Vape keyword filter (title/selftext) ---
KEYWORD_RE = re.compile(
    r"\b(vape|vaping|e-?cig|e\s*cigarette|e-?liquid|ejuice|"
    r"nic(?:otine)?\s*salt|pod(?:\s*system)?|disposable|mod|coil|atomizer|"
    r"lush\s*ice|strawberry\s*kiwi|blue\s*razz|mango\s*ice|elf\s*bar|juul|smok|vaporesso|relx|lost\s*mary)\b",
    flags=re.IGNORECASE
)

# --- 4) Simple flavor/product extraction (extend as you wish) ---
FLAVORS = [
    "Lush Ice", "Strawberry Kiwi", "Blue Razz", "Mango Ice", "Watermelon Ice",
    "Peach Ice", "Grape Ice", "Cool Mint", "Triple Mango", "Pink Lemonade", "Cola Ice"
]
BRANDS = ["Elf Bar", "JUUL", "SMOK", "Vaporesso", "RELX", "HQD", "Lost Mary", "Geek Bar", "OXBAR"]

def extract_product(text):
    hit = None
    for f in FLAVORS:
        if f.lower() in text.lower():
            hit = f
            break
    if not hit:
        for b in BRANDS:
            if b.lower() in text.lower():
                hit = b
                break
    return hit

def extract_hashtags(text):
    # Reddit rarely uses hashtags, but capture them if present
    return re.findall(r"#\w+", text)

def to_pk_time(utc_ts):
    dt = datetime.fromtimestamp(utc_ts, tz=timezone.utc).astimezone(ZoneInfo("Asia/Karachi"))
    return dt.strftime("%d/%m/%Y %H:%M:%S")  # matches your 23/08/2025 style

def fetch_reddit_vape_last_week(limit_per_sub=200):
    now_utc = datetime.now(timezone.utc)
    since_utc = now_utc - timedelta(days=7)
    since_ts = since_utc.timestamp()

    all_rows = []
    seen_ids = set()

    # Lucene query focuses search; we still post-filter with regex
    lucene_query = (
        '(title:(vape OR vaping OR "e cig" OR "e-cig" OR "e liquid" OR ejuice OR disposable OR "nic salt" OR pod)'
        ' OR selftext:(vape OR vaping OR "e cig" OR "e-cig" OR "e liquid" OR ejuice OR disposable OR "nic salt" OR pod))'
    )

    for sub in VAPE_SUBS:
        sr = reddit.subreddit(sub)
        # Search recent; we’ll still enforce the exact time range ourselves:
        for post in sr.search(lucene_query, syntax="lucene", sort="new", limit=limit_per_sub):
            if post.id in seen_ids:
                continue
            seen_ids.add(post.id)

            # Time window: last 7 days
            if not getattr(post, "created_utc", None):
                continue
            if post.created_utc < since_ts:
                continue

            # Text to filter against
            title = post.title or ""
            selftext = getattr(post, "selftext", "") or ""
            combined = f"{title}\n{selftext}"

            # Vape keyword filter
            if not KEYWORD_RE.search(combined):
                continue

            # Build row in your schema
            row = {
                "datetime": to_pk_time(post.created_utc),
                "niche": "vape",
                "product": extract_product(combined),
                "hashtags": extract_hashtags(combined),
                "description": title if len(title) > 5 else (selftext[:180] + "…"),
                "country": None,   # Reddit doesn't provide reliably
                "area": None,
                "store/channel_name": post.author.name if post.author else None,
                "social_platform": "reddit",
            }
            all_rows.append(row)

    return all_rows

if __name__ == "__main__":
    rows = fetch_reddit_vape_last_week()
    for r in rows[:10]:
        print(r)
    print(f"Total rows: {len(rows)}")


Total rows: 0


# Extract data from TikTokApi

In [24]:
from TikTokApi import TikTokApi